# 0. Data Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import numpy as np
from collections import Counter as ct
import datetime as dt

## Import from SQL & merge!(optional)

In [ ]:
cd /content/drive/MyDrive/archive/sql_2_csv

/content/drive/MyDrive/archive/sql_2_csv


In [ ]:
datas = os.listdir()

In [ ]:
datas

['academy.csv',
 'custom_workbook_problem.csv',
 'problem.csv',
 'problem_concept.csv',
 'problem_curriculum.csv',
 'problem_curriculum_concept_cache.csv',
 'public_workbook_problem.csv',
 'school_workbook_problem.csv',
 'student.csv',
 'student_workbook.csv',
 'student_workbook_progress.csv',
 'student_workbook_revision.csv',
 'student_worksheet.csv',
 'teacher.csv',
 'workbook.csv',
 'workbook_page.csv',
 'workbook_problem.csv',
 'worksheet.csv',
 'worksheet_problem.csv',
 'student_workbook_scoring.csv',
 'student_worksheet_scoring.csv']

In [ ]:
# int 64 -> 32
def int64to32(df):
    df[df.dtypes[df.dtypes == 'int64'].index] = df[df.dtypes[df.dtypes == 'int64'].index].astype(np.int32)
    return df
    
# read csv
def read_csv(df_name):
  # df_name을 변수명으로로
    globals()[df_name] =int64to32(pd.read_csv(df_name + ".csv").drop("Unnamed: 0",axis =1).rename(columns = {"id":df_name+"_id"}))
    

In [ ]:
# 이렇게 --.csv에서 --를 함수인자값으로 넣어주면, 해당 이름의 데이터프레임이 생성된다.

read_csv("academy")

In [ ]:
academy

,academy_id,status,create_datetime
0,D0000,WE_ARE_MATHFLAT,NaN
1,D0001,UNKNOWN,NaN
2,D0002,WE_ARE_MATHFLAT,NaN
3,D0004,PAY_INPROGRESS,2016-06-11 00:00:00
4,D0005,PAY_END,2016-06-11 00:00:00
...,...,...,...
18189,D9995,DELETED,2020-11-06 14:26:32
18190,D9996,PAY_INPROGRESS,2020-11-06 14:30:41
18191,D9997,PAY_DENY,2020-11-06 14:51:45
18192,D9998,PAY_INPROGRESS,2020-11-06 15:43:31


In [ ]:
# 사용할 데이터 목록
data_list = ["academy","student","worksheet", "student_worksheet", "worksheet_problem", 'student_worksheet_scoring', 'problem','problem_concept',"problem_curriculum",'problem_curriculum_concept_cache']

In [ ]:
# 이게 한 2분 정도 소요된다.

# 사용할 데이터 목록에 있는 각 이름을 'csv->dataframe'으로 만들기
for df_name in data_list:
    read_csv(df_name)

In [ ]:
student_worksheet

In [ ]:
# 주로 사용할 데이터프레임 = ['worksheet_problem', 'worksheet', 'student_worksheet', 'student_worksheet_scoring']

In [ ]:
# (worksheet, worksheet_problem) 합치기!

# (worksheet, worksheet_problem)에서 필요한 열; ['worksheet_problem_id', 'worksheet_id', 'school', 'grade', 'level', 'problem_id']
want_list = ['worksheet_problem_id', 'worksheet_id', 'school', 'grade', 'level', 'problem_id']

WtoWP = pd.merge(worksheet, worksheet_problem, left_on='worksheet_id', right_on='worksheet_id').loc[:, want_list]
WtoWP

In [ ]:
# (worksheet, worksheet_problem, student_worksheet_scoring) 합치기!

WtoWPtoSWS = pd.merge(WtoWP, student_worksheet_scoring, left_on='worksheet_problem_id', right_on='worksheet_problem_id')
WtoWPtoSWS

,worksheet_problem_id,worksheet_id,school,grade,level,problem_id,student_worksheet_scoring_id,student_worksheet_id,result,update_datetime
0,566528964,2052553,3,수학 I,3,350623,539233176,24946452,CORRECT,2022-06-06 20:54:51
1,566528965,2052553,3,수학 I,3,350621,539233177,24946452,CORRECT,2022-06-06 20:54:51
2,566528966,2052553,3,수학 I,3,350682,539233178,24946452,CORRECT,2022-06-06 20:54:51
3,566528967,2052553,3,수학 I,3,350653,539233179,24946452,CORRECT,2022-06-06 20:54:51
4,566528968,2052553,3,수학 I,3,350809,539233180,24946452,CORRECT,2022-06-06 20:54:51
...,...,...,...,...,...,...,...,...,...,...
94515274,649088276,23066616,3,수학 I,2,441117,649458405,27876029,NONE,2022-10-14 03:04:26
94515275,649088277,23066616,3,수학 I,2,343839,649458406,27876029,NONE,2022-10-14 03:04:26
94515276,649088278,23066616,3,수학 I,2,343814,649458407,27876029,NONE,2022-10-14 03:04:26
94515277,649088279,23066616,3,수학 I,2,344085,649458408,27876029,NONE,2022-10-14 03:04:26


In [ ]:
# (worksheet, student_worksheet) 합치기!

# (worksheet, student_worksheet)에서 필요한 열; ['worksheet_id', 'school', 'grade', 'level', 'student_worksheet_id', 'student_id', 'solve_datetime']
want_list = ['worksheet_id', 'school', 'grade', 'level', 'student_worksheet_id', 'student_id', 'solve_datetime']

WtoSW = pd.merge(worksheet, student_worksheet, left_on='worksheet_id', right_on='worksheet_id').loc[:, want_list]
WtoSW

,worksheet_id,school,grade,level,student_worksheet_id,student_id,solve_datetime
0,2052553,3,수학 I,3,2727913,I115673,2019-05-20 23:42:55
1,2052553,3,수학 I,3,2727914,I115675,2019-05-20 23:43:16
2,2052553,3,수학 I,3,2760547,I114726,NaN
3,2052553,3,수학 I,3,2760548,I114729,2019-05-27 21:25:11
4,2052553,3,수학 I,3,2760549,I114731,2019-05-25 19:37:36
...,...,...,...,...,...,...,...
3815065,23066391,3,수학 I,3,27875646,I306876,NaN
3815066,23066395,3,고등수학(하),3,27875649,I306876,NaN
3815067,23066600,3,고등수학(상),3,27876015,I824019,NaN
3815068,23066613,3,수학 II,1,27876025,I452302,NaN


In [ ]:
# (worksheet, student_worksheet, student_worksheet_scoring) 합치기!

WtoSWtoSWS = pd.merge(WtoSW, student_worksheet_scoring, left_on='student_worksheet_id', right_on='student_worksheet_id')
WtoSWtoSWS

,worksheet_id,school,grade,level,student_worksheet_id,student_id,solve_datetime,student_worksheet_scoring_id,worksheet_problem_id,result,update_datetime
0,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,539233176,566528964,CORRECT,2022-06-06 20:54:51
1,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,539233177,566528965,CORRECT,2022-06-06 20:54:51
2,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,539233178,566528966,CORRECT,2022-06-06 20:54:51
3,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,539233179,566528967,CORRECT,2022-06-06 20:54:51
4,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,539233180,566528968,CORRECT,2022-06-06 20:54:51
...,...,...,...,...,...,...,...,...,...,...,...
94515274,23066616,3,수학 I,2,27876029,I447015,NaN,649458405,649088276,NONE,2022-10-14 03:04:26
94515275,23066616,3,수학 I,2,27876029,I447015,NaN,649458406,649088277,NONE,2022-10-14 03:04:26
94515276,23066616,3,수학 I,2,27876029,I447015,NaN,649458407,649088278,NONE,2022-10-14 03:04:26
94515277,23066616,3,수학 I,2,27876029,I447015,NaN,649458408,649088279,NONE,2022-10-14 03:04:26


In [ ]:
# 최종적으로 (worksheet, worksheet_problem, student_worksheet, student_worksheet_scoring) 합치기!

result = pd.merge(WtoWPtoSWS, WtoSWtoSWS, left_on='student_worksheet_scoring_id', right_on='student_worksheet_scoring_id')
result

,worksheet_problem_id_x,worksheet_id_x,school_x,grade_x,level_x,problem_id,student_worksheet_scoring_id,student_worksheet_id_x,result_x,update_datetime_x,worksheet_id_y,school_y,grade_y,level_y,student_worksheet_id_y,student_id,solve_datetime,worksheet_problem_id_y,result_y,update_datetime_y
0,566528964,2052553,3,수학 I,3,350623,539233176,24946452,CORRECT,2022-06-06 20:54:51,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,566528964,CORRECT,2022-06-06 20:54:51
1,566528965,2052553,3,수학 I,3,350621,539233177,24946452,CORRECT,2022-06-06 20:54:51,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,566528965,CORRECT,2022-06-06 20:54:51
2,566528966,2052553,3,수학 I,3,350682,539233178,24946452,CORRECT,2022-06-06 20:54:51,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,566528966,CORRECT,2022-06-06 20:54:51
3,566528967,2052553,3,수학 I,3,350653,539233179,24946452,CORRECT,2022-06-06 20:54:51,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,566528967,CORRECT,2022-06-06 20:54:51
4,566528968,2052553,3,수학 I,3,350809,539233180,24946452,CORRECT,2022-06-06 20:54:51,2052553,3,수학 I,3,24946452,I801895,2022-06-06 20:54:51,566528968,CORRECT,2022-06-06 20:54:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94515274,649088276,23066616,3,수학 I,2,441117,649458405,27876029,NONE,2022-10-14 03:04:26,23066616,3,수학 I,2,27876029,I447015,NaN,649088276,NONE,2022-10-14 03:04:26
94515275,649088277,23066616,3,수학 I,2,343839,649458406,27876029,NONE,2022-10-14 03:04:26,23066616,3,수학 I,2,27876029,I447015,NaN,649088277,NONE,2022-10-14 03:04:26
94515276,649088278,23066616,3,수학 I,2,343814,649458407,27876029,NONE,2022-10-14 03:04:26,23066616,3,수학 I,2,27876029,I447015,NaN,649088278,NONE,2022-10-14 03:04:26
94515277,649088279,23066616,3,수학 I,2,344085,649458408,27876029,NONE,2022-10-14 03:04:26,23066616,3,수학 I,2,27876029,I447015,NaN,649088279,NONE,2022-10-14 03:04:26


In [ ]:
# 4분 정도 소요된다.

want_list = ['worksheet_id', 'school', 'grade', 'level', 'student_worksheet_id', 'student_worksheet_scoring_id', 'worksheet_problem_id', 'result', 'update_datetime']

result = pd.merge(WtoWPtoSWS, WtoSWtoSWS, left_on=want_list, right_on=want_list)
result

,worksheet_problem_id,worksheet_id,school,grade,level,problem_id,student_worksheet_scoring_id,student_worksheet_id,result,update_datetime,student_id,solve_datetime
0,566528964,2052553,3,수학 I,3,350623,539233176,24946452,CORRECT,2022-06-06 20:54:51,I801895,2022-06-06 20:54:51
1,566528965,2052553,3,수학 I,3,350621,539233177,24946452,CORRECT,2022-06-06 20:54:51,I801895,2022-06-06 20:54:51
2,566528966,2052553,3,수학 I,3,350682,539233178,24946452,CORRECT,2022-06-06 20:54:51,I801895,2022-06-06 20:54:51
3,566528967,2052553,3,수학 I,3,350653,539233179,24946452,CORRECT,2022-06-06 20:54:51,I801895,2022-06-06 20:54:51
4,566528968,2052553,3,수학 I,3,350809,539233180,24946452,CORRECT,2022-06-06 20:54:51,I801895,2022-06-06 20:54:51
...,...,...,...,...,...,...,...,...,...,...,...,...
94515274,649088276,23066616,3,수학 I,2,441117,649458405,27876029,NONE,2022-10-14 03:04:26,I447015,NaN
94515275,649088277,23066616,3,수학 I,2,343839,649458406,27876029,NONE,2022-10-14 03:04:26,I447015,NaN
94515276,649088278,23066616,3,수학 I,2,343814,649458407,27876029,NONE,2022-10-14 03:04:26,I447015,NaN
94515277,649088279,23066616,3,수학 I,2,344085,649458408,27876029,NONE,2022-10-14 03:04:26,I447015,NaN


In [ ]:
raw_result = result.copy()

want_list = ['student_id', 'school', 'grade', 'level', 'worksheet_id', 'solve_datetime', 'problem_id', 'result', 'update_datetime']
result = result.loc[:, want_list]
result

,student_id,school,grade,level,worksheet_id,solve_datetime,problem_id,result,update_datetime
0,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350623,CORRECT,2022-06-06 20:54:51
1,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350621,CORRECT,2022-06-06 20:54:51
2,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350682,CORRECT,2022-06-06 20:54:51
3,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350653,CORRECT,2022-06-06 20:54:51
4,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350809,CORRECT,2022-06-06 20:54:51
...,...,...,...,...,...,...,...,...,...
94515274,I447015,3,수학 I,2,23066616,NaN,441117,NONE,2022-10-14 03:04:26
94515275,I447015,3,수학 I,2,23066616,NaN,343839,NONE,2022-10-14 03:04:26
94515276,I447015,3,수학 I,2,23066616,NaN,343814,NONE,2022-10-14 03:04:26
94515277,I447015,3,수학 I,2,23066616,NaN,344085,NONE,2022-10-14 03:04:26


## 결과물과 저장

raw_result와 result가 결과물로 나와 이를 사용하면 되겠다.
두 데이터프레임을 저장하는 코드는 아래와 같으나, 용량이 너무 커서 램 부족으로 저장은 추천하지 않는다.

In [ ]:
raw_result.head()

In [ ]:
result.head()

In [ ]:
# 램 용량때문에 나눠서 저장하고, 다운로드 받아준다.

result.to_csv('/content/result.csv')

In [ ]:
# raw_result.to_csv('/content/raw_result.csv')

# 1. 학생별 '성장속도' 구하기

In [ ]:
result.head(2)

,student_id,school,grade,level,worksheet_id,solve_datetime,problem_id,result,update_datetime
0,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350623,CORRECT,2022-06-06 20:54:51
1,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350621,CORRECT,2022-06-06 20:54:51


In [ ]:
# 학생아이디 'I801895'에 대해 수행해보자!
id = 'I801895'

output = result[result.loc[:,'student_id'] == id]
output.head()

,student_id,school,grade,level,worksheet_id,solve_datetime,problem_id,result,update_datetime
0,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350623,CORRECT,2022-06-06 20:54:51
1,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350621,CORRECT,2022-06-06 20:54:51
2,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350682,CORRECT,2022-06-06 20:54:51
3,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350653,CORRECT,2022-06-06 20:54:51
4,I801895,3,수학 I,3,2052553,2022-06-06 20:54:51,350809,CORRECT,2022-06-06 20:54:51


In [ ]:
output = output.loc[output.loc[:,'result']!='NONE',['result', 'solve_datetime', 'update_datetime']]
output.head()

In [ ]:
a = output.iloc[:,:2]
a.head()

,result,solve_datetime
0,CORRECT,2022-06-06 20:54:51
1,CORRECT,2022-06-06 20:54:51
2,CORRECT,2022-06-06 20:54:51
3,CORRECT,2022-06-06 20:54:51
4,CORRECT,2022-06-06 20:54:51


In [ ]:
a.iloc[0,1]

'2022-06-06 20:54:51'

In [ ]:
a['solve_datetime'].sort_values

<bound method Series.sort_values of 0           2022-06-06 20:54:51
1           2022-06-06 20:54:51
2           2022-06-06 20:54:51
3           2022-06-06 20:54:51
4           2022-06-06 20:54:51
5           2022-06-06 20:54:51
6           2022-06-06 20:54:51
7           2022-06-06 20:54:51
8           2022-06-06 20:54:51
9           2022-06-06 20:54:51
10          2022-06-06 20:54:51
11          2022-06-06 20:54:51
12          2022-06-06 20:54:51
13          2022-06-06 20:54:51
14          2022-06-06 20:54:51
15          2022-06-06 20:54:51
16          2022-06-06 20:54:51
17          2022-06-06 20:54:51
58183210    2022-05-25 18:27:41
58183213    2022-05-25 18:27:41
58183216    2022-05-25 18:27:41
58183219    2022-05-25 18:27:41
58183222    2022-05-25 18:27:41
58183225    2022-05-25 18:27:41
58183228    2022-05-25 18:27:41
58183231    2022-05-25 18:27:41
58183234    2022-05-25 18:27:41
58183237    2022-05-25 18:27:41
58183240    2022-05-25 18:27:41
58183243    2022-05-25 18:27:41
5818